In [1]:
import os
import logging, os

logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [34]:
import tensorflow as tf
import keras
from musicnet.preprocessing.wav_specs_and_notes.utils import create_tf_record_ds
from musicnet.models.transformer.Transformer import F1FromSeqLogits, WeightedBinaryCrossentropy
from musicnet.utils import load_params, PROJECT_ROOT_DIR, notes_vocab
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
from utils import y_vs_y_pred_vis, spectogram_vis
from ipywidgets import interact
import librosa

In [3]:
model = keras.models.load_model(
    os.path.join(PROJECT_ROOT_DIR, "musicnet", "models", "transformer", "model.keras"),
    {
        "WeightedBinaryCrossentropy": WeightedBinaryCrossentropy,
        "F1FromSeqLogits": F1FromSeqLogits
    }
)

/home/leszek/projects/musicnet/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/leszek/projects/musicnet/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1721130044.101948  646248 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/test

In [21]:
params = load_params([
    "transformer.*",
    "wav_specs_and_notes.preprocessor.*",
    "wav_specs_and_notes.use_converted_midis"
])

In [5]:
ds_params = {
    "architecture": params["architecture"],
    "n_filters": params["n_filters"],
    "target_classes": model.get_config()["target_classes"],
    "batch_size": params["batch_size"],
    "dataset_size": params["dataset_size"],
    "use_converted_midis": params["use_converted_midis"]
}

In [6]:
train_ds = create_tf_record_ds("train", **ds_params)
val_ds = create_tf_record_ds("val", **ds_params)

In [7]:
model.evaluate(val_ds)

I0000 00:00:1721130048.171719  646749 service.cc:146] XLA service 0x7fad24005070 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721130048.171746  646749 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Ti, Compute Capability 8.6


      5/Unknown 13s 31ms/step - f1_score: 0.3383 - loss: 1.9544

I0000 00:00:1721130060.173914  646749 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


125/125 ━━━━━━━━━━━━━━━━━━━━ 17s 28ms/step - f1_score: 0.2913 - loss: 2.3415


/usr/local/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


[2.2519478797912598, 0.28854188323020935]

In [8]:
for x, y in val_ds:
    break

In [9]:
print(x.shape, y.shape)

(8, 1000, 200) (8, 1000, 83)


In [14]:
y_pred = model(x)
y_pred = tf.sigmoid(y_pred)

In [22]:
spectogram_params = { k: v for k, v in params.items() if k in ["n_fft", "target_sr", "min_hz", "n_filters"] }
spectogram_params

{'target_sr': 44100, 'n_fft': 2205, 'min_hz': 0, 'n_filters': 200}

In [62]:
spectogram_params = { k: v for k, v in params.items() if k in ["n_fft", "target_sr", "min_hz"] }

@interact(i=(0, params["batch_size"] - 1), t=(0.5, 1.0))
def show_sample(i, t=0.5):
    y_vs_y_pred_vis(y[i].numpy(), (y_pred > t)[i].numpy())
    spectogram_vis(x[i].numpy(), **spectogram_params)

interactive(children=(IntSlider(value=3, description='i', max=7), FloatSlider(value=0.5, description='t', max=…

In [217]:
# @tf.function
# def run_inference(context, max_len=999, head_start=0):
#     y_pred = tf.TensorArray(dtype=tf.bool, size=0, dynamic_size=True)
#     for i in tf.range(head_start):
#         y_pred = y_pred.write(i, y[-1, i, :])
#     for i in tf.range(head_start, max_len):
#         next_pred = model([context, [y_pred.stack()]], training=False)[-1, -1, :]
#         next_pred = tf.sigmoid(next_pred)
#         y_pred = y_pred.write(i, next_pred > 0.5)
#     return y_pred.stack()

In [251]:
# y_pred_infer = run_inference(context, y.shape[1], head_start=100)
# y_pred_context = model([context, x])[0]
# y_pred_context = tf.sigmoid(y_pred_context).numpy() > 0.5